In [ ]:
# 6. Gruppierung & Aggregation

## 1. Bibliotheken importieren und Daten laden

In [29]:
# Bibliotheken importieren
import pandas as pd
import os

# Pfade definieren
base_path = r"C:\Users\faink\Desktop\Achievement_4\02_Daten"
raw_path = os.path.join(base_path, "Rohe_Daten")
prepared_path = os.path.join(base_path, "Prepared_Daten")

# DataFrame laden (angenommen als Pickle-Datei aus der vorherigen Übung)
ords_prods_merge = pd.read_pickle(os.path.join(prepared_path, "ords_prods_merge.pkl"))

# products.csv laden (enthält product_id, product_name, aisle_id, department_id, evtl. price)
products = pd.read_csv(os.path.join(prepared_path, "products_checked.csv"))

# Überblick über die Daten
ords_prods_merge.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,2539329,1,prior,1,2,8,NaN,196,1,0
1,2539329,1,prior,1,2,8,NaN,14084,2,0
2,2539329,1,prior,1,2,8,NaN,12427,3,0
3,2539329,1,prior,1,2,8,NaN,26088,4,0
4,2539329,1,prior,1,2,8,NaN,26405,5,0


## 2. Aggregierter Mittelwert von „ordernumber“ pro „departmentid“ (gesamter DataFrame)

In [24]:
# Department-Datei laden
departments = pd.read_csv(os.path.join(prepared_path, "departments_wrangled.csv"))

# Überblick über die Daten
departments.head()

,Unnamed: 0,index,department
0,0,1,frozen
1,1,2,other
2,2,3,bakery
3,3,4,produce
4,4,5,alcohol


## 3. Loyalitäts-Flag erstellen

In [25]:
# Loyalitäts-Flag erstellen
ords_prods_merge['loyalty_flag'] = ords_prods_merge.groupby('user_id')['order_number'] \
    .transform(lambda x: 'Loyal Customer' if x.max() > 40 else ('Regular Customer' if x.max() > 10 else 'New Customer'))

# Prüfen
ords_prods_merge[['user_id', 'order_number', 'loyalty_flag']].head(10)

,user_id,order_number,loyalty_flag
0,1,1,New Customer
1,1,1,New Customer
2,1,1,New Customer
3,1,1,New Customer
4,1,1,New Customer
5,1,2,New Customer
6,1,2,New Customer
7,1,2,New Customer
8,1,2,New Customer
9,1,2,New Customer


## 4. Produktpreis-Statistiken nach Loyalität

In [26]:
ords_prods_merge.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,loyalty_flag
0,2539329,1,prior,1,2,8,NaN,196,1,0,New Customer
1,2539329,1,prior,1,2,8,NaN,14084,2,0,New Customer
2,2539329,1,prior,1,2,8,NaN,12427,3,0,New Customer
3,2539329,1,prior,1,2,8,NaN,26088,4,0,New Customer
4,2539329,1,prior,1,2,8,NaN,26405,5,0,New Customer


## 5. Ausgaben-Flag erstellen

In [31]:
products.head()

,product_id,product_name,aisle_id,department_id,prices
0,1,Chocolate Sandwich Cookies,61,19,5.8
1,2,All-Seasons Salt,104,13,9.3
2,3,Robust Golden Unsweetened Oolong Tea,94,7,4.5
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1,10.5
4,5,Green Chile Anytime Sauce,5,13,4.3


In [36]:
# Mit ords_prods_merge zusammenführen
ords_prods_merge = ords_prods_merge.merge(products[['product_id','prices']], on='product_id', how='left')

# Prüfen
ords_prods_merge[['user_id','product_id','prices']].head()

,user_id,product_id,prices
0,1,196,9.0
1,1,14084,12.5
2,1,12427,4.4
3,1,26088,4.7
4,1,26405,1.0


In [37]:
# Durchschnittlicher Produktpreis pro Nutzer
avg_price_per_user = ords_prods_merge.groupby('user_id')['prices'].mean().reset_index()
avg_price_per_user['spending_flag'] = avg_price_per_user['prices'].apply(lambda x: 'Low spender' if x < 10 else 'High spender')

# DataFrame zusammenführen
ords_prods_merge = ords_prods_merge.merge(avg_price_per_user[['user_id','spending_flag']], on='user_id', how='left')

# Prüfen
ords_prods_merge[['user_id','prices','spending_flag']].head(10)


,user_id,prices,spending_flag
0,1,9.0,Low spender
1,1,12.5,Low spender
2,1,4.4,Low spender
3,1,4.7,Low spender
4,1,1.0,Low spender
5,1,9.0,Low spender
6,1,3.0,Low spender
7,1,4.4,Low spender
8,1,10.3,Low spender
9,1,4.7,Low spender


## 6. Bestellhäufigkeits-Flag erstellen

In [38]:
# Median days_since_prior_order pro Nutzer
median_days = ords_prods_merge.groupby('user_id')['days_since_prior_order'].median().reset_index()

def frequency_flag(days):
    if days > 20:
        return 'Nicht-häufiger Kunde'
    elif days > 10:
        return 'Regelmäßiger Kunde'
    else:
        return 'Häufiger Kunde'

median_days['frequency_flag'] = median_days['days_since_prior_order'].apply(frequency_flag)

# DataFrame zusammenführen
ords_prods_merge = ords_prods_merge.merge(median_days[['user_id','frequency_flag']], on='user_id', how='left')

# Prüfen
ords_prods_merge[['user_id','days_since_prior_order','frequency_flag']].head(10)

,user_id,days_since_prior_order,frequency_flag
0,1,NaN,Nicht-häufiger Kunde
1,1,NaN,Nicht-häufiger Kunde
2,1,NaN,Nicht-häufiger Kunde
3,1,NaN,Nicht-häufiger Kunde
4,1,NaN,Nicht-häufiger Kunde
5,1,15.0,Nicht-häufiger Kunde
6,1,15.0,Nicht-häufiger Kunde
7,1,15.0,Nicht-häufiger Kunde
8,1,15.0,Nicht-häufiger Kunde
9,1,15.0,Nicht-häufiger Kunde


## 7. DataFrame exportieren 

In [39]:
# Als Pickle-Datei speichern
ords_prods_merge.to_pickle(os.path.join(prepared_path, "ords_prods_merge_prepared.pkl"))